In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies=movies.merge(credits,on="title")

In [ ]:
#genres
#id
#keywords
#overview
#title
#cast
#crew

movies=movies[['id','title','overview','genres','keywords','cast','crew']]


In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True) # Remove rows with missing values

In [ ]:
movies.duplicated().sum() # Check for duplicates

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L    


In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)
movies.head()

In [ ]:
def convert_cast(obj):
    L = []
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 5:  # Limit to first 5 cast members
            L.append(i['name'])
            counter += 1
        else:
            break
    return L  

In [ ]:
movies['cast']=movies['cast'].apply(convert_cast)

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x: x.split())  # Tokenize the overview
movies.head()

In [ ]:
movies['genre']=movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])  # Remove spaces in genres
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x]) 
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x]) 
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x]) 

In [ ]:
movies['tags'] =movies['overview']+ movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))  # Join the tags into a single string

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())  # Convert to lowercase

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps= PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)  # Apply stemming to the tags
cv.get_feature_names_out()  # Get feature names after stemming

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
import pickle
pickle.dump(new_df, open('movies.pkl', 'wb'))  # Save the DataFrame to a pickle file

In [135]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))  # Save the similarity matrix to a pickle file